In [1]:
from DataSplit import load_data
from DataSplit import tokenize

X_train, X_test, X_eval, y_train, y_test, y_eval = load_data()


tokenize(X_train) # Basic tokenizer that removes punctuation and lowercases everything



In [ ]:
from Tokenizer import remove_stopwords_and_stem

for x in range(len(X_train)):
    X_train[x] = remove_stopwords_and_stem(X_train[x])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ellio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: 'int' object is not iterable